In [2]:
import numpy as np
from numpy import loadtxt
import pickle

from flask import Flask,render_template,url_for,request

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# load model
model = load_model('sentiment_analysis.h5')

In [4]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [25]:
def predict_review(model, new_sentences, max_length=100, show_padded_sequence=True ):
    # Keep the original sentences so that we can keep using them later
      # Create an array to hold the encoded sequences
    result_dict = dict()
    result_dict['Input'] = new_sentences
    if isinstance(new_sentences, str):
        new_sentences = [' '.join(new_sentences.split(" "))]
    trunc_type='post' 
    padding_type='post'

    # text to sequence
    new_text_sequences = tokenizer.texts_to_sequences(new_sentences)

    # Pad all sequences for the new reviews
    new_reviews_padded = pad_sequences(new_text_sequences, maxlen=max_length, 
                                     padding=padding_type, truncating=trunc_type)             

    classes = model.predict(new_reviews_padded)

    # The closer the class is to 1, the more positive the review is
    for x in range(len(new_sentences)):
        
        # Print its predicted class
        if classes[x] > 0.5:
            result_dict['Result'] = "Positive"
            #print("Positive")
        else:
            result_dict['Result'] = "Negative"
            #print("Negative")
        result_dict['Probability'] = classes[x]
    
    #return result_dict['Input'], result_dict['Input'], result_dict['Probability']
    return result_dict

In [34]:
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():
    if request.method == 'POST':
        message = request.form['message']
        
        my_prediction = predict_review(model, message, show_padded_sequence=False)
        #seq, pred, prob = predict_review(model, message, show_padded_sequence=False)
    return render_template('result.html', result_dict = my_prediction)



if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/May/2021 17:24:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2021 17:24:57] "GET /static/css/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [21/May/2021 17:25:02] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2021 17:25:02] "GET /static/css/styles.css HTTP/1.1" 404 -
